# Chapter 2 Persistence

Functional data structures need to have persistent property to be useful to functional programmers, which means update to them requires *copying* to preserve the old values. *Sharing* is the design technique to minimize the time and space cost of copying.

## Lists

A list can be used to implement a stack, queue, deque, or a concatenable list. First define the interface

In [1]:
exception Empty
exception Subscript

module type Stack = sig
  type 'a t

  val empty: 'a t
  val isEmpty: 'a t -> bool
  
  val cons: 'a -> 'a t -> 'a t
  val head: 'a t -> 'a             (* raises Empty *)
  val tail: 'a t -> 'a t       (* raises Empty *)
  
  val (++): 'a t -> 'a t -> 'a t
  val update: 'a t -> int -> 'a -> 'a t       (* raises Subscript *)
end

exception Empty


exception Subscript


module type Stack =
  sig
    type 'a t
    val empty : 'a t
    val isEmpty : 'a t -> bool
    val cons : 'a -> 'a t -> 'a t
    val head : 'a t -> 'a
    val tail : 'a t -> 'a t
    val ( ++ ) : 'a t -> 'a t -> 'a t
    val update : 'a t -> int -> 'a -> 'a t
  end


this can be implemented in various ways.

In [2]:
module NativeList : Stack = struct
  type 'a t = 'a list
  
  let empty = []
  let isEmpty s = s = []
  
  let cons x s = x :: s
  let head s = try List.hd s
               with exn -> raise Empty
  let tail s = try List.tl s
               with exn -> raise Empty
               
  (*  Add update  *)
  let rec update s i x = match (s, i, x) with
    | ([], _, _) -> raise Subscript
    | (y::ys, 0, x) -> x::ys
    | (y::ys, n, x) -> y::(update ys (n-1) x)
  
  let (++) = List.append
end

module NativeList : Stack


In [3]:
module CustomList : Stack = struct
  type 'a t = Nil | Cons of 'a * 'a t
  
  let empty = Nil
  let isEmpty s = s = Nil
  
  let cons x s = Cons (x, s)
  let head = function
    | Nil -> raise Empty
    | Cons (x, _) -> x
  let tail = function
    | Nil -> raise Empty
    | Cons (_, s) -> s
    
  (*  Add Update  *)
  let rec update s i x = match (s, i, x) with
    | (Nil, _, _) -> raise Subscript
    | (Cons (y, ys), 0, x) -> Cons (x, ys)
    | (Cons (y, ys), n, x) -> Cons (y, update ys (n-1) x)
  
  let rec (++) xss yss = match (xss, yss) with
    | (_, Nil) -> xss
    | (Nil, _) -> yss
    | (Cons (x, xs), _) -> Cons (x, (xs ++ yss))
end

module CustomList : Stack


The most important part is the implementation of Update

### Excercise 2.1

Write a function `suffixes: 'a stack -> 'a stack stack` that takes a list `xs` and returns a list of all the suffixes of `xs` in descending order of length

In [4]:
let rec suffixes = function
  | [] -> [[]]
  | hd::tl as xs -> xs::(suffixes tl)

val suffixes : 'a list -> 'a list list = <fun>


In [5]:
suffixes [1;2;3;4]

- : int list list = [[1; 2; 3; 4]; [2; 3; 4]; [3; 4]; [4]; []]


## Trees

In [6]:
module type Set = sig
  type e
  type t
  
  val empty: t
  val insert: e -> t -> t
  val member: e -> t -> bool
end

module type Set =
  sig
    type e
    type t
    val empty : t
    val insert : e -> t -> t
    val member : e -> t -> bool
  end


In [7]:
module type Ordered = sig
  type t

  val compare : t -> t -> int
end

module type Ordered = sig type t val compare : t -> t -> int end


In [8]:
module UnbalancedSst (Elem : Ordered) : (Set with type e = Elem.t) =
struct
  type e = Elem.t
  type t = Empty | Tree of t * e * t
  
  let empty = Empty
  let rec member x = function
    | Empty -> false
    | Tree (l, y, r) -> match Elem.compare x y with
      | 0 -> true
      | n when n < 0 -> member x l
      | _ -> member x r
      
  let rec insert x = function
    | Empty -> Tree (Empty, x, Empty)
    | Tree (l, y, r) as s -> match Elem.compare x y with
      | 0 -> s
      | n when n < 0 -> Tree (insert x l, y, r)
      | _ -> Tree (l, y, insert x r)
end

module UnbalancedSst :
  functor (Elem : Ordered) ->
    sig
      type e = Elem.t
      type t
      val empty : t
      val insert : e -> t -> t
      val member : e -> t -> bool
    end
